In [3]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import csv
import numpy as np
import pandas as pd

# Paths for datasets
dataset_paths = {
    "aaai": "/Users/ashutosh/Downloads/GeoS/aaai",
    "official": "/Users/ashutosh/Downloads/GeoS/official",
    "practice": "/Users/ashutosh/Downloads/GeoS/practice"
}

# Load CSV with extracted text-based shape labels
csv_file = "GeoS_Processed.csv"
df = pd.read_csv(csv_file)

# Function to detect shapes in an image
def detect_shape(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
    
    # Check if the image was loaded properly
    if img is None:
        print(f"❌ Error: Could not read image {image_path}")
        return "unknown"  # Return "unknown" instead of crashing

    blurred = cv2.GaussianBlur(img, (5,5), 0)  # Reduce noise
    edges = cv2.Canny(blurred, 50, 150)  # Detect edges

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        num_vertices = len(approx)

        # Shape classification based on vertices
        if num_vertices == 3:
            return "triangle"
        elif num_vertices == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            return "square" if 0.95 <= aspect_ratio <= 1.05 else "rectangle"
        elif num_vertices == 5:
            return "pentagon"
        elif num_vertices == 6:
            return "hexagon"
    
    # Check for circles using Hough Transform
    circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=10, maxRadius=100)
    if circles is not None:
        return "circle"

    return "unknown"  # If no shape detected

# Initialize accuracy metrics
TP, FP, FN, TN = 0, 0, 0, 0

# Process each dataset folder
for dataset_name, dataset_path in dataset_paths.items():
    if not os.path.exists(dataset_path):
        print(f"❌ Error: Folder '{dataset_path}' not found! Skipping...")
        continue

    # Filter CSV data for current dataset
    dataset_df = df[df["dataset"] == dataset_name]

    # Iterate through JSON files (which correspond to image files)
    for _, row in dataset_df.iterrows():
        json_file = row["file_name"]
        expected_shape = row["shape"]
        image_file = json_file.replace(".json", ".png")
        image_path = os.path.join(dataset_path, image_file)

        if not os.path.exists(image_path):
            print(f"⚠️ Image '{image_file}' not found, skipping...")
            continue

        # Detect shape from the image
        detected_shape = detect_shape(image_path)

        # Compare detected vs. expected shape
        if expected_shape != "unknown":  # Only evaluate when ground truth is known
            if detected_shape == expected_shape:
                TP += 1  # Correct detection
            else:
                FP += 1  # Wrong detection
        elif detected_shape != "unknown":
            FP += 1  # False positive (should not detect but did)
        else:
            TN += 1  # Correctly detected as unknown

# Compute Accuracy
total = TP + FP + FN + TN
accuracy = (TP + TN) / total * 100 if total > 0 else 0

# Print Results
print("\n🔹 Accuracy Results:")
print(f"✅ True Positives (Correct Detections): {TP}")
print(f"❌ False Positives (Wrong Detections): {FP}")
print(f"⚠️ False Negatives (Missed Shapes): {FN}")
print(f"✅ True Negatives (Correctly Ignored): {TN}")
print(f"\n🎯 **Final Accuracy: {accuracy:.2f}%**")

❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/050.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/046.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/027.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/026.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/047.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/051.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/060.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/021.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/056.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/001.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/017.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/040.png
❌ Error: Could not read image /Users/ashutosh/Downloads/GeoS/aaai/041.png
❌ Error: Could not read image /Users/a

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile



🔹 Accuracy Results:
✅ True Positives (Correct Detections): 10
❌ False Positives (Wrong Detections): 160
⚠️ False Negatives (Missed Shapes): 0
✅ True Negatives (Correctly Ignored): 15

🎯 **Final Accuracy: 13.51%**


In [4]:
import os
import cv2
import csv

# Manually specify dataset paths
dataset_paths = {
    "aaai": "/Users/ashutosh/Downloads/GeoS/aaai",
    "official": "/Users/ashutosh/Downloads/GeoS/official",
    "practice": "/Users/ashutosh/Downloads/GeoS/practice"
}

# CSV output file
output_csv = "unreadable_images.csv"

# Store unreadable images
unreadable_images = []

# Process each dataset folder
for dataset_name, dataset_path in dataset_paths.items():
    if not os.path.exists(dataset_path):
        print(f"❌ Error: Folder '{dataset_path}' not found! Skipping...")
        continue

    # Iterate through files in the dataset folder
    for file in os.listdir(dataset_path):
        if file.endswith(".png"):  # Only check image files
            image_path = os.path.join(dataset_path, file)

            # Try reading the image
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"❌ OpenCV cannot read the image: {image_path}")
                unreadable_images.append([dataset_name, file, image_path])

# Save unreadable images to CSV
with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["dataset", "file_name", "image_path"])  # CSV header
    writer.writerows(unreadable_images)

print(f"✅ Unreadable images saved to {output_csv}")

❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/002.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/016.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/017.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/003.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/029.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/015.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/001.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/028.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/004.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/038.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/013.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/012.png
❌ OpenCV cannot read the image: /Users/ashutosh/Downloads/GeoS/aaai/049.png
❌ OpenCV can

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [ ]:
appoxpolyDP

## For images that are readable by the

In [5]:
import os
import cv2
import csv

# Manually specify dataset paths
dataset_paths = {
    "aaai": "/Users/ashutosh/Downloads/GeoS/aaai",
    "official": "/Users/ashutosh/Downloads/GeoS/official",
    "practice": "/Users/ashutosh/Downloads/GeoS/practice"
}

# CSV output file
output_csv = "readable_images.csv"

# Store readable images
readable_images = []

# Process each dataset folder
for dataset_name, dataset_path in dataset_paths.items():
    if not os.path.exists(dataset_path):
        print(f"❌ Error: Folder '{dataset_path}' not found! Skipping...")
        continue

    # Iterate through files in the dataset folder
    for file in os.listdir(dataset_path):
        if file.endswith(".png"):  # Only check image files
            image_path = os.path.join(dataset_path, file)

            # Try reading the image
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if img is not None:
                print(f"✅ Readable image: {image_path}")
                readable_images.append([dataset_name, file, image_path])

# Save readable images to CSV
with open(output_csv, "w", newline="") as f:
    
    writer = csv.writer(f)
    writer.writerow(["dataset", "file_name", "image_path"])  # CSV header
    writer.writerows(readable_images)

print(f"✅ Readable images saved to {output_csv}")

✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/000.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/014.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/010.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/039.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/005.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/011.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/007.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/006.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/058.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/065.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/043.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/037.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/036.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/008.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/034.png
✅ Readable image: /Users/ashutosh/Downloads/GeoS/aaai/035.png
✅ Readab

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [8]:
import os
import cv2
import csv
import warnings
import numpy as np
from PIL import Image

# Suppress OpenCV warnings
cv2.setLogLevel(0)  # ✅ Works on all OpenCV versions
warnings.filterwarnings("ignore")

# Manually specify dataset paths
dataset_paths = {
    "aaai": "/Users/ashutosh/Downloads/GeoS/aaai",
    "official": "/Users/ashutosh/Downloads/GeoS/official",
    "practice": "/Users/ashutosh/Downloads/GeoS/practice"
}

# CSV output file
output_csv = "processed_images.csv"

# Store processed images (both readable and fixed ones)
processed_images = []

# Function to fix PNG files before reading
def fix_png(image_path):
    try:
        img = Image.open(image_path)
        img.save(image_path)  # Overwrite with fixed version
    except Exception as e:
        print(f"❌ Failed to fix image: {image_path}, Error: {e}")

# Process each dataset folder
for dataset_name, dataset_path in dataset_paths.items():
    if not os.path.exists(dataset_path):
        print(f"❌ Error: Folder '{dataset_path}' not found! Skipping...")
        continue

    # Iterate through files in the dataset folder
    for file in os.listdir(dataset_path):
        if file.endswith(".png"):  # Only check image files
            image_path = os.path.join(dataset_path, file)

            # Fix the PNG before reading
            fix_png(image_path)

            # Try reading the image
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if img is not None:
                print(f"✅ Processed image: {image_path}")
                processed_images.append([dataset_name, file, image_path])
            else:
                print(f"⚠️ OpenCV could not read image even after fixing: {image_path}")

# Save processed images to CSV
with open(output_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["dataset", "file_name", "image_path"])  # CSV header
    writer.writerows(processed_images)

print(f"✅ All images processed and saved to {output_csv}")

✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/002.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/016.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/017.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/003.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/029.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/015.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/001.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/000.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/014.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/028.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/010.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/004.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/038.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/039.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/005.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/aaai/

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/045.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/051.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/050.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/044.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/023.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/037.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/036.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/022.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/008.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/034.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/020.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/021.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/035.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/009.png
✅ Processed image: /Users/ashutosh/Downloads/GeoS/official/031